## Математический аппарат логистической регрессии

<img src='../img/logreg1.png'>


<img src='../img/logreg2.png'>


## Загрузка и предобработка данных

In [1]:
# загружаем необходимые библиотеки
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import display
plt.rc('font', family='Verdana')

In [2]:
# загружаем данные
data = pd.read_csv("C:/Trees/Bankloan.csv", encoding='cp1251', sep=';')
data.head()

,age,employ,address,debtinc,creddebt,default
0,28,7,2,"17,7","2,990592",0
1,64,34,17,"14,7","5,047392",0
2,40,20,12,"4,8","1,042368",0
3,30,11,3,"34,5","1,75122",0
4,25,2,2,"22,4","0,75936",1


In [3]:
# заменяем запятые на точки и переводим в тип float
for i in ['debtinc', 'creddebt']:
    if i in data.columns:
        data[i]=data[i].str.replace(',', '.').astype('float')

In [4]:
# выводим первые 5 наблюдений
data.head()

,age,employ,address,debtinc,creddebt,default
0,28,7,2,17.7,2.990592,0
1,64,34,17,14.7,5.047392,0
2,40,20,12,4.8,1.042368,0
3,30,11,3,34.5,1.751220,0
4,25,2,2,22.4,0.759360,1


In [5]:
# преобразовываем в тип object
data['default']=data['default'].astype('object')

In [6]:
# выводим информацию о переменных
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 6 columns):
age         1500 non-null int64
employ      1500 non-null int64
address     1500 non-null int64
debtinc     1500 non-null float64
creddebt    1500 non-null float64
default     1500 non-null object
dtypes: float64(2), int64(3), object(1)
memory usage: 70.4+ KB


## Дамми-кодирование

In [7]:
# выполняем дамми-кодирование
print("Исходные переменные:\n", list(data.columns), "\n")
data_dummies = pd.get_dummies(data)
print("Переменные после get_dummies:\n", list(data_dummies.columns))

Исходные переменные:
 ['age', 'employ', 'address', 'debtinc', 'creddebt', 'default'] 

Переменные после get_dummies:
 ['age', 'employ', 'address', 'debtinc', 'creddebt', 'default_0', 'default_1']


## Создание обучающего и тестового массивов признаков и меток

In [8]:
# создаем массив меток и массив признаков
y = data_dummies.loc[:, 'default_1']
data_dummies.drop('default_0', axis=1, inplace=True)
data_dummies.drop('default_1', axis=1, inplace=True)
X = data_dummies.loc[:, 'age':'creddebt']

In [9]:
# создаем обучающий и тестовый массивы
# признаков и меток
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## Стандартизация

Важнейшей предпосылкой регрессионного анализа является единый масштаб измерения переменных. Давайте отмасштабируем переменные. Самое простое масштабирование подразумевает, что из каждого значения переменной мы вычтем среднее значение и полученный результат разделим на стандартное отклонение:

$$\frac{\Large x_i - mean(x)}{\Large stdev(x)}$$
 
В итоге мы получаем распределение со средним 0 и стандартным отклонением 1. Именно это и делает класс `StandardScaler`. Сначала импортируем класс `StandardScaler`, который осуществляет предварительную обработку, а затем создаем его экземпляр.

In [10]:
# импортируем класс StandardScaler
from sklearn.preprocessing import StandardScaler
# создаем экземпляр класса StandardScaler
scaler = StandardScaler()

Затем с помощью метода `fit` мы подгоняем `scaler` на обучающих данных. В отличие от обычных моделей машинного обучения, при вызове метода `fit` `scaler` работает с данными (`X_train`), а ответы (`y_train`) не используются.

In [11]:
# подгоняем модель
scaler.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

Чтобы применить преобразование, которое мы только что подогнали, то есть фактически отмасштабировать (scale) обучающие и контрольные данные, мы воспользуемся методом `transform`. Метод `transform` используется в `scikit-learn`, когда модель возвращает новое представление данных. Обратите внимамние, мы всегда применяем одинаковое преобразование к обучающему и тестовому наборам. Это означает, что метод `transform` всегда вычитает среднее значение, вычисленное для обучающего набора, и делит на стандартное отклонение, вычисленное также для обучающего набора. Среднее значение и стандартное отклонение переменной для обучающего набора могут отличаться от среднего значения и стандартного отклонения переменной для тестового набора.

In [12]:
# преобразовываем данные и по отмасштабированным данным
# уже можно будет строить логистическую регрессию
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [13]:
# давайте посмотрим минимальное и максимальное значения каждого 
# признака в отмасштабированных данных
print("Минимальное значение для каждого признака\n{}".format(X_train_scaled.min(axis=0)))
print("Максимальное значение для каждого признака\n {}".format(X_train_scaled.max(axis=0)))

Минимальное значение для каждого признака
[-1.22953373 -0.78904361 -1.05315752 -1.48834012 -0.64472937]
Максимальное значение для каждого признака
 [  3.3466552    6.36785134   4.53924518   4.60164867  11.2239057 ]


## Построение логистической регрессии

In [14]:
# импортируем функцию roc_auc_score
from sklearn.metrics import roc_auc_score

# импортируем класс LogisticRegression
from sklearn.linear_model import LogisticRegression

# строим модель логистической регрессии на данных,
# отмасштабированных с помощью RobustScaler
logreg = LogisticRegression().fit(X_train_scaled, y_train)
print("AUC на обучающей выборке: {:.3f}".
      format(roc_auc_score(y_train, logreg.predict_proba(X_train_scaled)[:, 1])))
print("AUC на контрольной выборке: {:.3f}".
      format(roc_auc_score(y_test, logreg.predict_proba(X_test_scaled)[:, 1])))

AUC на обучающей выборке: 0.848
AUC на контрольной выборке: 0.832


## Вычисление бета-коэффициентов и экспоненциальных коэффициентов логистической регрессии

In [15]:
# выводим бета-коэффициенты
logreg.coef_

array([[-0.43709748, -1.93336605,  0.01066257,  0.68409413,  1.46949422]])

In [16]:
# запишем бета-коэффициенты и названия предикторов
# в отдельные объекты
coef=logreg.coef_
feat_labels = X.columns

In [17]:
# вычислим свободный член (константу)
intercept=logreg.intercept_
intercept

array([-1.04780638])

In [18]:
# печатаем название "Бета-коэффициенты"
print("Бета-коэффициенты:")
# для удобства сопоставим каждому названию 
# предиктора соответствующий бета-коэффициент
for c, feature in zip(coef[0], feat_labels):
    print(feature, round(c, 2))

Бета-коэффициенты:
age -0.44
employ -1.93
address 0.01
debtinc 0.68
creddebt 1.47


In [19]:
# вычислим экспоненциальные коэффициенты
# и запишем их в отдельный объект
exp_coef=np.exp(coef)
exp_coef

array([[ 0.64590847,  0.14466044,  1.01071961,  1.98197561,  4.34703593]])

In [20]:
# печатаем название "Экспоненциальные коэффициенты"
print("Экспоненциальные коэффициенты:")
# для удобства сопоставим каждому названию 
# предиктора соответствующий 
# экспоненциальный коэффициент
for c, feature in zip(exp_coef[0], feat_labels):
    print(feature, round(c, 2))

Экспоненциальные коэффициенты:
age 0.65
employ 0.14
address 1.01
debtinc 1.98
creddebt 4.35


## Интерпретация коэффициентов логистической регрессии

<img src='../img/logreg3.png'>

<img src='../img/logreg4.png'>


## Получение прогнозов

In [21]:
# получаем спрогнозированные значения
predvalue = logreg.predict(X_train_scaled)
predvalue

array([1, 0, 0, ..., 1, 0, 0], dtype=uint8)

In [22]:
# преобразуем массив NumPy со спрогнозированными
# значениями в объект DataFrame
predvalue=pd.DataFrame(predvalue, index=X_train.index, columns=['Predicted_class'])
predvalue.head()

,Predicted_class
485,1
527,0
199,0
889,0
844,1


In [23]:
# получаем спрогнозированные вероятности
prob = logreg.predict_proba(X_train_scaled)
prob

array([[ 0.49572801,  0.50427199],
       [ 0.7785396 ,  0.2214604 ],
       [ 0.64167471,  0.35832529],
       ..., 
       [ 0.23013997,  0.76986003],
       [ 0.95370803,  0.04629197],
       [ 0.97289379,  0.02710621]])

In [24]:
# преобразуем массив NumPy со спрогнозированными
# вероятностями в объект DataFrame
probabilities=pd.DataFrame(prob, index=X_train.index, columns=['Prob0', 'Prob1'])
probabilities.head()

,Prob0,Prob1
485,0.495728,0.504272
527,0.778540,0.221460
199,0.641675,0.358325
889,0.801448,0.198552
844,0.428586,0.571414


In [25]:
# преобразуем массив NumPy с отмасштабированными
# данными в объект DataFrame
X_tr=pd.DataFrame(X_train_scaled, index=X_train.index, columns=feat_labels)
X_tr.head()

,age,employ,address,debtinc,creddebt
485,-1.004475,-0.675442,-0.888675,-0.228342,-0.350020
527,-0.479339,-0.334638,-0.230745,-0.573342,-0.456745
199,-0.104241,-0.334638,-0.395228,-0.078343,-0.115391
889,2.746499,-0.561841,1.085114,0.011657,-0.172074
844,-1.079495,-0.789044,-0.888675,-0.033343,-0.428474


In [26]:
# конкатенируем датафреймы
result=pd.concat([X_tr, y_train, predvalue, probabilities], axis=1)
# выводим первые 5 наблюдений
# итогового датафрейма
result.head()

,age,employ,address,debtinc,creddebt,default_1,Predicted_class,Prob0,Prob1
485,-1.004475,-0.675442,-0.888675,-0.228342,-0.350020,0,1,0.495728,0.504272
527,-0.479339,-0.334638,-0.230745,-0.573342,-0.456745,0,0,0.778540,0.221460
199,-0.104241,-0.334638,-0.395228,-0.078343,-0.115391,0,0,0.641675,0.358325
889,2.746499,-0.561841,1.085114,0.011657,-0.172074,0,0,0.801448,0.198552
844,-1.079495,-0.789044,-0.888675,-0.033343,-0.428474,0,1,0.428586,0.571414


## Вычисление вероятности положительного класса вручную

Давайте вручную вычислим вероятность дефолта для наблюдения 485. 

Подставляем в формулу

$$\Large \frac{1}{1+e^{-(b_0+b_1x_i^{(1)}+b_2x_i^{(2)}+...+b_kx_i^{(k)})}}$$ 

коэффициенты и значения предикторов для данного наблюдения.

<img src='../img/Prob_logreg.png'>


Найденное значение практически совпадает со значением вероятности дефолта, вычисленным автоматически (0.504341). Небольшое различие обусловлено ошибкой округления.

## Вычисление значений решающей функции

In [27]:
# выводим значения решающей функции
# для первых 5 наблюдений
logreg.decision_function(X_train_scaled[:5])

array([ 0.01708838, -1.25717605, -0.5826403 , -1.39536818,  0.2876232 ])

Возвращаемое значение представляет собой число с плавающей точкой для каждого наблюдения.Значение показывает, насколько сильно модель уверена в том, что точка данных принадлежит положительному классу, в данном случае, классу 1. Положительное значение указывает на предпочтение в пользу позиционного класса, а отрицательное значение – на предпочтение в пользу отрицательного (другого) класса. Мы можем судить о прогнозах, лишь взглянув на знак решающей функции.

In [28]:
# возвращаем логические значения в соответствии с условием
# "значение решающей функции выше 0" и прогнозы, 
# они будут идентичны друг другу
print(logreg.decision_function(X_train_scaled[:5]) > 0)
print(logreg.predict(X_train_scaled[:5]))

[ True False False False  True]
[1 0 0 0 1]
